Assignment 3 notes 

In [3]:
#import packages 
import os
import pandas as pd 
import gensim.downloader as api

#packages for install
#pandas gensim

Flow of assignment (in one script):

- Loads the song lyric data
- Downloads/loads a word embedding model via ```gensim``` (see below)

- Takes a given word as an input and finds the most similar words via word embeddings
- Find how many songs for a given artist feature terms from the expanded query
- Calculate the percentage of that artist's songs featuring those terms
- Print and/or save results in an easy-to-understand way
    - For example, "45% of {ARTIST}'s songs contain words related to {SEARCH TERM}"

In [4]:
#load songs
filenpath = os.path.join("in", "Spotify Million Song Dataset_exported.csv")

songs = pd.read_csv(filenpath)

In [5]:
#load model
model = api.load("glove-wiki-gigaword-50")

[==================================================] 100.0% 66.0/66.0MB downloaded


In [6]:
# Function to check if artists exists in corpus, makes both input and corpus lower case 
def artist_find(artist_q):
    """
    The function takes a string, if the string matches with an artist name, 
    a subset of the songs dataframe is returned, only with the given artist's songs.
    """
    artist_q = artist_q.lower()
    songs["artist"] = (songs["artist"]).str.lower()
    
    if (songs['artist'].eq(artist_q).any()):
        artist_songs = songs[songs["artist"] == artist_q]
    else: print('Artist not found') 

    return artist_songs

In [11]:
ts_songs = artist_find("Taylor Swift")

In [8]:
query = model.most_similar("love")
query_ls = ([i[0] for i in query])
query_ls.append("love")

In [15]:
ts_songs["text"]

19137    You have a way of coming easily to me  \nAnd w...
19138    I've never gone with the wind, just let it flo...
19139    All I knew this morning when I woke  \nIs I kn...
19140    You take a deep breath  \nAnd you walk through...
19141    Hey Stephen, I know looks can be deceiving  \n...
                               ...                        
52781    He said the way my blue eyes shined  \nPut tho...
52782    Today was a fairytale  \nYou were the prince  ...
52783    I'm dreaming of a white Christmas  \nJust like...
52784    Say you're sorry  \nThat face of an angel  \nC...
52785    He said, "Let's get out of this town,  \nDrive...
Name: text, Length: 81, dtype: object

In [9]:
ts_text = list(ts_songs["text"])

In [35]:
song_count = 0
for text in ts_songs["text"]:
    text_tokens = text.lower().split()

    q_count = 0
    for keyword in query_ls:
        q_count = q_count + (text_tokens.count(keyword))

if q_count > 0:
    song_count =+1
    print(text[1])
    

e


In [22]:
(ts_songs["text"]).iloc[2]

"All I knew this morning when I woke  \nIs I know something now, know something now I didn't before  \nAnd all I've seen since 18 hours ago is green eyes and freckles  \nAnd your smile in the back of my mind making me feel like  \nI just want to know you better, know you better, know you better now  \nI just want to know you better, know you better, know you better now  \nI just want to know you better, know you better, know you better now  \nI just want to know, you know you know you  \n  \n'Cause all I know is we said hello  \nAnd your eyes look like coming home  \nAll I know is a simple name, everything has changed  \nAll I know is you held the door  \nYou'll be mine and I'll be yours  \nAll I know since yesterday is everything has changed  \n  \nAnd all my walls stood tall painted blue  \nBut I'll take them down, take them down and open up the door for you  \n  \nAnd all I feel in my stomach is butterflies the beautiful kind  \nMaking up for lost time, taking flight, making me feel

In [104]:
text = ts_songs["text"].iloc[0]
text_tokens = text.lower().split()

In [115]:
song_count = 0

for 
for i in query_ls:
    q_count = text_tokens.count(query_ls[i])
    if q_count> 0 :
        song_count =+1


TypeError: list.count() takes exactly one argument (0 given)

In [112]:
q_count = text_tokens.count(['you','cold'])